In [0]:
bronze_base = "/Volumes/aqi_etl_pipeline/bronze_aqi/bronze_vol_aqi/aqi/"


# ==========================================================
# STEP 2 — Find latest YEAR
# ==========================================================
years = [item.name.replace("/", "") for item in dbutils.fs.ls(bronze_base)]
years_int = [int(y) for y in years if y.isdigit()]
latest_year = str(max(years_int))

print("Latest Year:", latest_year)


# ==========================================================
# STEP 3 — Find latest MONTH
# ==========================================================
months = [item.name.replace("/", "") for item in dbutils.fs.ls(f"{bronze_base}/{latest_year}")]
months_int = [int(m) for m in months if m.isdigit()]
latest_month = str(max(months_int)).zfill(2)

print("Latest Month:", latest_month)


# ==========================================================
# STEP 4 — Find latest DAY
# ==========================================================
days = [item.name.replace("/", "") for item in dbutils.fs.ls(f"{bronze_base}/{latest_year}/{latest_month}")]
days_int = [int(d) for d in days if d.isdigit()]
latest_day = str(max(days_int)).zfill(2)

print("Latest Day:", latest_day)


# ==========================================================
# STEP 5 — Find latest HOUR
# ==========================================================
hours = [item.name.replace("/", "") for item in dbutils.fs.ls(f"{bronze_base}/{latest_year}/{latest_month}/{latest_day}")]
hours_int = [int(h) for h in hours if h.isdigit()]
latest_hour = str(max(hours_int)).zfill(2)

print("Latest Hour:", latest_hour)


# ==========================================================
# STEP 6 — Find latest FILE inside that hour folder
# ==========================================================
files = dbutils.fs.ls(f"{bronze_base}/{latest_year}/{latest_month}/{latest_day}/{latest_hour}")

# Extract epoch from file names: weather_<epoch>.json
file_epochs = []
file_map = {}  # epoch → full path

for f in files:
    name = f.name  # example: weather_1763967555.json
    if name.startswith("weather_") and name.endswith(".json"):
        epoch = int(name.replace("weather_", "").replace(".json", ""))
        file_epochs.append(epoch)
        file_map[epoch] = f.path

latest_epoch = max(file_epochs)
latest_file_path = file_map[latest_epoch]

print("Latest File:", latest_file_path)

In [0]:
df_bronze = spark.read.json(latest_file_path)

print("Bronze schema:")
df_bronze.printSchema()

In [0]:
from pyspark.sql import functions as F

df_flat = df_bronze.select(
    F.col("data.aqi").alias("AQI"),
    F.col("data.city.name").alias("City"),
    F.col("data.city.geo").getItem(0).alias("latitude"),
    F.col("data.city.geo").getItem(1).alias("longitude"),
    F.col("data.time.s").alias("timestamp"),
    F.col("ingested_at").alias("ingested_at")
)

display(df_flat)

In [0]:
df_silver = (
    df_flat

    # Convert weather dt_raw → UTC timestamp
    .withColumn("weather_utc", F.from_unixtime(F.col("dt_raw")))

    # Convert weather timestamp → IST using timezone offset
    .withColumn("weather_ist",
        F.from_unixtime(F.col("dt_raw") + F.col("tz_offset_seconds"))
    )

    # Convert ingestion timestamp string → timestamp (UTC)
    .withColumn("ingested_at_utc", F.to_timestamp("ingested_at_raw"))

    # ingestion time in IST (+5:30)
    .withColumn("ingested_at_ist",
        F.expr("ingested_at_utc + INTERVAL 5 HOURS 30 MINUTES")
    )
)
